# CSE527 Homework 3 - Part 3
**Due date: 23:59 on Nov 26, 2023**


## Description
---
Segmentation has been recently gaining more popularity in the field of computer vision. In this homework you will be implementing a standard UNet model to perform semantic segmentation. While transofrmer based architectures have demostrated impressive perfromance in semantic/instance segmentation, it's important to be familiar with other architectures commonly used for semantic segmentation.

You will use the same dataset that you have used in part 2 of this homework, but the output of the model here is a class level segmentation mask.

Input of the model will be an RGB image (3xHxW) and output will be a boolean mask (HxW).

| Input Image      | Output Mask (class:horse) |
| ----------- | ----------- |
| ![Input Image](https://drive.google.com/uc?id=1BqmNxzV4t5MFQHjBFizlI1p9xlEatXiz)| ![Output Image](https://drive.google.com/uc?id=1NCW95D8JDBNsgzfFNa14ZiV_SiCpBPIv)|

  
Read the paper: [U-Net: Convolutional Networks for Biomedical Image Segmentation
](https://arxiv.org/abs/1505.04597) to understand the structure of UNet models, what they are composed of, and how to build them.  

Also you should refer to this github repo to understand the implementations of a UNet: [milesial/Pytorch-UNet](https://github.com/milesial/Pytorch-UNet) . Specifically refer to unet/unet_model.py to gain understanding of the convolutional blocks, and skip connections.

In this part you will be designing UNet architecture simmilar to it but with specific changes.

## Code
---


In [1]:
HW3_ROOT_PATH = '/gdrive/My Drive/' + 'MUPPARAPU_SAIKOUSHIK_114999629_hw3/' #'YOUR HOMEWORK3 PATH HERE'
PATH_TO_PART3 = HW3_ROOT_PATH + 'part3/'
PATH_TO_COCO = HW3_ROOT_PATH + 'coco/'


You will be using this code [unet.zip](https://drive.google.com/file/d/1p2tMMUfF2vfcr3QHipJx0XyiBOMWoMNM/view?usp=sharing) to complete your work. This code contains all the tools for loading dataset, training the newtorks, loss methods and evaluation metrics.


To pull this code into your drive, you have to first add the zip file as shortcut to current working directory (PATH_TO_PART3 above). To do this, open the above link and click on "Add shortcut to Drive" button (drive symbol with a plus) and navigate to working PATH_TO_PART3 directory and add shortcut.


We will use the same dataset that you have used in the Part 2 : [coco.zip dataset](https://drive.google.com/file/d/1GVyxYHwVgiG9z_Sn46wslT_2n65DLZRw/view?usp=sharing). Put this in the root of your hw3 HW3_ROOT_PATH. If you have this extracted to some path you can re-use the it.


In [2]:
from google.colab import drive
drive.mount('/gdrive')
# CD into root of your homework2 part3 directory
%cd -q $HW3_ROOT_PATH
!mkdir -p '{PATH_TO_PART3}'
%cd $PATH_TO_PART3

Mounted at /gdrive
/gdrive/My Drive/MUPPARAPU_SAIKOUSHIK_114999629_hw3/part3


In [3]:
!ls

CSE527_23F_HW3_P3.ipynb  unet


In [ ]:
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-crf0b77a
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-crf0b77a
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
  Created wheel for pycocotools: filename=pycocotools-2.0-cp310-cp310-linux_x86_64.whl size=375457 sha256=2fa68431642ff82e80df37330d616bdbc9498d8a581f92ef3d5153263346dfa4
  Stored in directory: /tmp/pip-ephem-wheel-cache-levy2zix/wheels/39/61/b4/480fbddb4d3d6bc34083e7397bc6f5d1381f79acc68e9f3511
Successfully built pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.7
    Uninstalling pycocotools-2.0.7:
      Successfully uninstalled pycocotools-2.0.7


In [ ]:
!unzip -n ../coco.zip -d ..
!unzip -n unet.zip

Archive:  unet.zip
  inflating: unet/coco.py            
  inflating: unet/main.py            
  inflating: unet/transforms.py      
  inflating: unet/unet.py            
   creating: unet/util/
 extracting: unet/util/__init__.py   
  inflating: unet/util/box_ops.py    
  inflating: unet/util/misc.py       
  inflating: unet/util/plot_utils.py  


In [4]:
%cd -q unet
!ls

coco.py		       images	__pycache__	    transforms.py  util
concat_checkpoint.pth  main.py	sum_checkpoint.pth  unet.py


### Load helper method

In [ ]:
import matplotlib.pyplot as plt
import os
import numpy as np

def show_images(folder_path):
    image_paths = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    image_paths = image_paths[:50]
    rows = 5
    cols = 10
    fig, axes = plt.subplots(rows, cols, figsize=(15, 8))
    for i, image_path in enumerate(image_paths):
        image = plt.imread(image_path)
        row = int(i / cols)
        col = i % cols
        axes[row, col].imshow(image)
        axes[row, col].set_title(os.path.basename(image_path))
        axes[row, col].axis('off')
    plt.tight_layout()
    plt.show()

## Training
---
You will use main.py for training, evaluating and visualizing.



In [ ]:
!python main.py --help

device cuda
usage: main.py [-h] [--coco_path COCO_PATH] [--batch_size BATCH_SIZE] [--epochs EPOCHS] [--eval]
               [--load_pretrained] [--skip {concat,sum}] [--checkpoint CHECKPOINT]

Train a segmentation model on COCO data.

options:
  -h, --help            show this help message and exit
  --coco_path COCO_PATH
                        Path to COCO dataset directory.
  --batch_size BATCH_SIZE
                        Batch size for training.
  --epochs EPOCHS       Epochs for training.
  --eval                Whether to evaluate the model on the validation set.
  --load_pretrained     Whether to evaluate the model on the validation set.
  --skip {concat,sum}   Skip connection strategy: either concatenation (concat) or summation
                        (sum).
  --checkpoint CHECKPOINT
                        Path to checkpoint file.


##Implement UNet (15 points)
---


`unet.py` contains two classes `UNet()` and `SumUNet()`, both are implementations of `torch.nn.Module()`.

You need to complete the methods `UNet:__init__()` and `UNet:forward()` to implement a segmentation architecture that processes input of shape `N x 3 x H x W` (N is batch size) and will output a mask of shape `N x 1 x H x W`

###Requirements
1. Encoder part of the UNet will be made of pytorch Resnet's layers 1 to 4.

    https://pytorch.org/vision/main/_modules/torchvision/models/resnet.html

    Essentially, your UNet's encoder will be using `torchvision.models.resnet`'s `layer1, layer2, layer3, layer4` for downsampling.

2. When argument `load_pretrained_encoder_layers` of __init__() is set to true, the layers will be initialized with the pretrained weights. You can use the default pre-trained weights provided by pytorch.

3. As described in the paper or above repo, `concatenation` operations will be done for the skip connections. (in the upcoming section, you will instead use summation)


Note:
- You are recommended to size your model under 25M paramenters. As more parameters will hurt the trainig time and even effeciency.
- For the decoder section (later half), you will not be able to use any pre-trained weights, so you will use randomly initialized blocks (that you will design on your own). Each block usually have layers of `[Conv, BatchNorm, ReLu]`
- You need not use any ReLU on the last mask output layer.

Hint:
- chose a [resnet](https://pytorch.org/vision/main/_modules/torchvision/models/resnet.html) size
- understand the `In` channels & `Out` channels of each layer of your chosen resnet()
- build decoder layers so that they will work along the above 4 layers
- conclude how to use the skip connections. ( you will using `concatenation`).
- complete the forward() method
- use print(x.shape) statements to understand the flow of tensors

Log reference: [log](https://drive.google.com/file/d/1ZGd-ciOKkajri0GEOwc-uJDBjGJHJdNY/view?usp=drive_link)

Aim for mIoU more than 0.35. You may not be penalized for low perfromance of your model.  



In [6]:
!python main.py --skip concat --coco_path "{PATH_TO_COCO}"

device cuda
Namespace(coco_path='/gdrive/My Drive/MUPPARAPU_SAIKOUSHIK_114999629_hw3/coco/', batch_size=20, epochs=20, eval=False, load_pretrained=False, skip='concat', checkpoint='concat_checkpoint.pth')
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
loading annotations into memory...
Done (t=0.47s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Epoch: [0]  [ 0/25]  eta: 0:04:00    time: 9.6248  data: 7.0852  max mem: 8653
Epoch: [0]  [10/25]  eta: 0:01:49    time

### Evaluation
To just run the evaluation or to produce the predicted masks in the `images/` folder, you need to run the main.py with --eval.
Note that this will load the latest stored checkpoint corresponding to the --skip flag that you have provided.

In [7]:
!python main.py --skip concat --coco_path "{PATH_TO_COCO}" --eval

device cuda
Namespace(coco_path='/gdrive/My Drive/MUPPARAPU_SAIKOUSHIK_114999629_hw3/coco/', batch_size=20, epochs=20, eval=True, load_pretrained=False, skip='concat', checkpoint='concat_checkpoint.pth')
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Performance on validation dataset: mIoU:0.3574138581752777 , mDice:0.48272210359573364


### With --load_pretrained

When properly configured using pre-trained weights should result in higher mIoUs. You can expect mIoU around .65  

In [ ]:
!python main.py --skip concat --coco_path "{PATH_TO_COCO}"  --load_pretrained

device cuda
Namespace(coco_path='/gdrive/My Drive/MUPPARAPU_SAIKOUSHIK_114999629_hw3/coco/', batch_size=20, epochs=20, eval=False, load_pretrained=True, skip='concat', checkpoint='concat_checkpoint.pth')
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100% 44.7M/44.7M [00:00<00:00, 48.8MB/s]

In [ ]:
!python main.py --skip concat --coco_path "{PATH_TO_COCO}" --eval

device cuda
Namespace(coco_path='/gdrive/My Drive/MUPPARAPU_SAIKOUSHIK_114999629_hw3/coco/', batch_size=20, epochs=20, eval=True, load_pretrained=False, skip='concat', checkpoint='concat_checkpoint.pth')
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Performance on validation dataset: mIoU:0.6403228640556335 , mDice:0.738150954246521


Let us see the masks generated by the best run:
It is okay if the masks are not perfectly aligned. We are only training over 500 images for a short duration!

In [ ]:
folder_path = 'images'
show_images(folder_path)

Output hidden; open in https://colab.research.google.com to view.

##Implement SumUNet (10 points)
---
This is very much similar to above UNet, but for the skip connections we will be using summation operation instead of concatenation. This is kind of an open ended question, in the sense that there are multuple ways you can design connections.

Feel free to use any layers | blocks | up/down sampling.


Hint:
- Again you need to focus on the feature shapes
- you can use sampling techniques to change the height/width
- you can use conv layers to change number of channels  

Note: You can chose to remove --load_pretrained from the below cell if you think that will have a better perfromance


In [ ]:
!python main.py --skip sum --coco_path "{PATH_TO_COCO}"  --load_pretrained

device cuda
Namespace(coco_path='/gdrive/My Drive/MUPPARAPU_SAIKOUSHIK_114999629_hw3/coco/', batch_size=20, epochs=20, eval=False, load_pretrained=True, skip='sum', checkpoint='sum_checkpoint.pth')
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Epoch: [0]  [

In [ ]:
!python main.py --skip sum --coco_path "{PATH_TO_COCO}"  --eval

device cuda
Namespace(coco_path='/gdrive/My Drive/MUPPARAPU_SAIKOUSHIK_114999629_hw3/coco/', batch_size=20, epochs=20, eval=True, load_pretrained=False, skip='sum', checkpoint='sum_checkpoint.pth')
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Performance on validation dataset: mIoU:0.6314481496810913 , mDice:0.7304691672325134


## Submission guidelines
---



The submission structure should look like:

IMPORTANT: Remove the images/ folder, and any CHECKPOINT files.


```
{last name}_{first name}_{sbu id}_hw2/
├── part1/
│   └── ...
├── part2/
│   └── ...
├── part3/
│   ├── CSE527_23F_HW3_P3.ipynb
│   └── unet/
│       ├── utils/
│       │   ├── ...
│       │   └── ...
│       ├── coco.py
│       ├── main.py
│       ├── transforms.py
│       └── unet.py
```


Follow instructions in part1's submission guidelines to generate your complete submission